In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import string
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import emoji


In [2]:
tweet= pd.read_csv('train.csv')
test_tw=pd.read_csv('test.csv')
test_tw.shape

(3263, 4)

In [3]:
#удаляю дубликаты из train.csv

In [4]:
#поиск дубликатов с разными targets. Раскомментировать, если нужно пересчитать 
#tweet["tt"] = tweet["target"].astype("str").str.cat(tweet["text"], sep = " ", na_rep=' ')
#mask = tweet["text"].duplicated(keep = False) 
#mask2 = tweet[mask]["tt"].duplicated(keep = False)
#print(df[mask & mask1 & -mask2][60:80])

remove_ids = [353, 390, 872, 875, 1771, 1773, 4051,4076, 4080, 4092, 4499, 4507, 4918, 4935, 
6012, 6017, 6217, 6219, 6220, 6223, 6366, 6382, 6775, 6788, 7362, 7386, 8018, 8044, 7602, 7626,
8296, 8297, 9499, 9501, 796, 822]
#print(df[df["id"] == remove_ids[0]])
#print(df[df["id"] == remove_ids[0]].iloc[0,0])
print(len(remove_ids))

for i in remove_ids:
    rm = tweet[tweet["id"] == i].index[0]
    #print(df.iloc[rm])
    tweet = tweet.drop([rm])
    print(i, rm, tweet.shape)



In [5]:
df=pd.concat([tweet,test_tw], sort="False")
df.shape
tweet.shape

(7613, 5)

In [6]:
url = re.compile(r'https?://\S+|www\.\S+')

def get_URL(text):
    u = re.findall(url, text)
    return u if u else ''  

def remove_URL(text):
    return re.sub(url, r"", text)

def remove_HTML(text):
    soup = BeautifulSoup(text)
    return soup.get_text()

# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
#emoji_pattern = re.compile("["
#                           u"\U0001F600-\U0001F64F"  # emoticons
#                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                           u"\U00002702-\U000027B0"
#                           u"\U000024C2-\U0001F251"
#                           "]+", flags=re.UNICODE)
emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F""]+", flags=re.UNICODE)


def get_emoji(text):
    u = re.findall(emoji_pattern, text)
    return u

def emoji2text(text):
    em = re.search(emoji_pattern, text)
    if not(em is None):
        u = emoji.demojize(text)
        print(u)
        return u
    else:
        return text

def emoji2text2(text):
    return emoji.demojize(text)

def remove_emoji(text):
    return emoji_pattern.sub(r'', text)

def get_tags(x):
    return {str(tag.strip("#")) for tag in x.split() if tag.startswith("#")}


In [7]:
#df['text_original'] = df["text"]
#df['text'] = df['text'].apply(lambda x: remove_HTML(x))

In [8]:
#предположим, что url пригодится, как отдельный признак. От url можно title раздобыть
df['url'] = df['text'].apply(lambda x: get_URL(x))
#df['text'] = df['text'].apply(lambda x: remove_URL(x))

In [9]:
#предположим, что emoji пригодятся, как категориальный признак
#df["emoji"] = df['text'].apply(lambda x: get_emoji(x))
#df['text'] = df['text'].apply(lambda x: remove_emoji(x))

In [10]:
#кажется смайликов там нет. 
#df['text'] = df['text'].apply(lambda x: emoji2text2(x))
#f = open('data_deemoji.csv', 'w')
#df.to_csv(f)

In [11]:
#вытаскиваем хештеги
#df["tags"] =  df['text'].apply(lambda x: get_tags(x))

In [12]:
#убираем пунктуацию - в этой версии оставим
#df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))


In [13]:
# клеим текст, location, keyword
df["text"] = df["text"].str.cat(df["keyword"], sep = " ", na_rep=' ').str.cat(df["location"], sep = " ", na_rep=' ')


In [14]:
df[245:250]

,id,keyword,location,target,text,url
245,348,annihilation,NaN,0.0,Evildead - Annihilation of Civilization http:/...,[http://t.co/sPfkE5Kqu4]
246,349,annihilation,NaN,0.0,U.S National Park Services Tonto National Fore...,[http://t.co/6LoJOoROuk]
247,352,annihilation,Wild Wild Web,1.0,annihilating quarterstaff of annihilation anni...,
248,353,annihilation,Subconscious LA,0.0,World Annihilation vs Self Transformation http...,"[http://t.co/pyehwodWun, http://t.co/pB2N77nSKz]"
249,354,annihilation,Spain,1.0,:StarMade: :Stardate 3: :Planetary Annihilatio...,[http://t.co/I2hHvIUmTm]


from spellchecker import SpellChecker
spell = SpellChecker()
list = ['wildfire', 'wildfires', 'rockyfire', 'ebay', 'coworker', 'youtube', 'hailstorm', 'instagram', 'ebola', 'iphone']
for l in list:
    spell.word_frequency.add(l)

def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    if len(misspelled_words) > 0:
        print(misspelled_words)
        for word in misspelled_words:
            text = re.sub(word, spell.correction(word), text)
            #print(text)
    return text

#print("resp:", correct_spellings("corect my nam, ples"))
        
df['text']=df['text'].apply(lambda x : correct_spellings(x))


In [15]:
df.shape

(10876, 6)

In [16]:
#закавычивание строк с запятыми внутри"
df["text"].apply(lambda x: x if re.findall(",",x) is None else "".join(['"',x,'"']))


0       "Our Deeds are the Reason of this #earthquake ...
1            "Forest fire near La Ronge Sask. Canada    "
2       "All residents asked to 'shelter in place' are...
3       "13,000 people receive #wildfires evacuation o...
4       "Just got sent this photo from Ruby #Alaska as...
                              ...                        
3258    "EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FAST...
3259    "Storm in RI worse than last hurricane. My cit...
3260    "Green Line derailment in Chicago http://t.co/...
3261    "MEG issues Hazardous Weather Outlook (HWO) ht...
3262    "#CityofCalgary has activated its Municipal Em...
Name: text, Length: 10876, dtype: object

In [17]:
df.shape

(10876, 6)

In [18]:
df["text"].loc[10813]= "@stighefootball Begovic has been garbage. He got wrecked by a Red Bull reserve team and everyone else this preseason wrecked Brussels, Belgium"

In [22]:
f = open('data_proc3.csv', 'w')
sep = ","
df.to_csv(f, sep=sep)
f.close
test = pd.read_csv('data_proc3.csv', sep=sep)
print(test[test["target"].isna()].shape)
print(test.shape)

(3263, 7)
(10876, 7)
